In [62]:
# Ελεγχος version pyspark .. Θέλουμε το 3.4.1
import pyspark
print(pyspark.__version__)

3.4.1


In [63]:
# add java to path .. δεν μπορουσα να βρω αλλον τροπο να τρέξει η spark.
# (θελουμε την Java 17 επειδη μονο η Java 8 , Java 11 και Java 17 δουλευει με το pyspark)
import os
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-17"
os.environ["PATH"] += ";C:\\Program Files\\Java\\jdk-17\\bin"

In [64]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import month, col, desc, year, max

In [65]:
# Φτιάχνουμε ενα spark session
spark = SparkSession.builder.appName("StockAnalysis").getOrCreate()

In [66]:
# Διαβάζουμε τα δεδομένα

agn = "data/agn.us.txt"
ainv = "data/ainv.us.txt"
ale = "data/ale.us.txt"

agn_data = spark.read.csv(agn, header=True)
ainv_data = spark.read.csv(ainv, header=True)
ale_data = spark.read.csv(ale, header=True)

In [67]:
# Ελεγχος αν διαβαστηκαν σωστα τα αρχεια
print(agn_data.show())

+----------+------+------+------+------+-------+-------+
|      Date|  Open|  High|   Low| Close| Volume|OpenInt|
+----------+------+------+------+------+-------+-------+
|2005-01-03| 32.31| 32.31|31.527|31.616|1027044|      0|
|2005-01-04|31.527|31.616| 31.22|31.338|1927762|      0|
|2005-01-05|30.971|31.051|30.714|30.843| 943399|      0|
|2005-01-06|30.843|31.398|30.764| 31.26| 662398|      0|
|2005-01-07| 31.26| 31.26|30.456|30.566|1087886|      0|
|2005-01-10|30.506| 31.23|30.486|30.863| 896381|      0|
|2005-01-11|30.843|30.903|30.199|30.377| 835841|      0|
|2005-01-12|30.456|30.466|29.872| 30.07|1476041|      0|
|2005-01-13| 30.07|30.605|29.882|30.129| 772578|      0|
|2005-01-14|30.189|30.337| 30.07|30.239| 453538|      0|
|2005-01-18| 30.09|30.724|29.862|30.496| 592979|      0|
|2005-01-19|30.506|30.605|29.852| 30.08| 694685|      0|
|2005-01-20| 30.06|30.407| 30.05|  30.1| 858644|      0|
|2005-01-21|30.129|30.307|29.584|29.723|1040463|      0|
|2005-01-24|29.763|29.931|29.28

In [68]:
# Ελεγχος αν διαβαστηκαν σωστα τα αρχεια
print(ainv_data.show())

+----------+------+------+------+------+-------+-------+
|      Date|  Open|  High|   Low| Close| Volume|OpenInt|
+----------+------+------+------+------+-------+-------+
|2005-02-25|7.9331|8.0665|7.8034|8.0665| 632096|      0|
|2005-02-28|8.0756|8.1248|7.8524|7.8615| 750398|      0|
|2005-03-01|7.8673|8.1101|7.8524|8.0756| 977229|      0|
|2005-03-02|8.0385|8.1335|8.0385|8.0625| 837318|      0|
|2005-03-03|8.0625| 8.148|8.0287|8.0996| 541465|      0|
|2005-03-04|8.0707| 8.148|7.9183|8.1429|1219409|      0|
|2005-03-07|8.1429|8.2104|8.0756|8.1248| 587457|      0|
|2005-03-08|8.1676|8.2162|8.0813|8.0813|1124808|      0|
|2005-03-09|8.2051|8.2051|8.0385|8.0625| 503940|      0|
|2005-03-10|8.1156|8.2162|8.0756|8.1284| 369083|      0|
|2005-03-11|8.0863|8.1719|8.0756|8.1719| 393268|      0|
|2005-03-14|8.1719|8.2436|8.1284|8.2394| 540575|      0|
|2005-03-15|8.2735|8.5727|8.2623|8.4196| 884438|      0|
|2005-03-16|8.4196|8.4354|8.2248|8.2539| 495105|      0|
|2005-03-17|8.2333| 8.462|8.157

In [69]:
# Ελεγχος αν διαβαστηκαν σωστα τα αρχεια
print(ale_data.show())

+----------+------+------+------+------+------+-------+
|      Date|  Open|  High|   Low| Close|Volume|OpenInt|
+----------+------+------+------+------+------+-------+
|2005-02-25|30.717|31.104|  30.6|31.087|214817|      0|
|2005-02-28|31.073|31.087|30.652|  30.7|180957|      0|
|2005-03-01|30.659|30.946|30.606|30.946|223738|      0|
|2005-03-02|30.908|31.179|30.884|31.073|238730|      0|
|2005-03-03|31.073|31.211|30.987|31.195|224381|      0|
|2005-03-04|31.335|31.836|31.288|31.836|227100|      0|
|2005-03-07|31.875|33.788|31.836|33.663|676255|      0|
|2005-03-08|33.624|34.352|33.492|33.663|716061|      0|
|2005-03-09|33.588|33.595|32.834|32.899|501505|      0|
|2005-03-10|32.921|33.146|32.573|32.873|329596|      0|
|2005-03-11|32.873| 32.89|32.687|32.796| 91254|      0|
|2005-03-14|32.796|33.152|32.719|32.842|302839|      0|
|2005-03-15|32.907|33.338| 32.89|33.338|443985|      0|
|2005-03-16|33.269|33.278|32.687|32.742|237567|      0|
|2005-03-17|32.742|32.921|32.742|32.915| 96942| 

In [70]:
# Μετατροπή των στηλών σε float

def cast_column_to_numeric(df, column):
    return df.withColumn(column, col(column).cast("float"))

In [71]:
# Μετατροπή των στηλών σε float
columns_to_cast = ["Open", "Close", "Volume"]
for col_name in columns_to_cast:
    agn_data = cast_column_to_numeric(agn_data, col_name)
    ainv_data = cast_column_to_numeric(ainv_data, col_name)
    ale_data = cast_column_to_numeric(ale_data, col_name)

In [72]:
# Μετατροπη των στηλων 'Open' και 'Date' σε αριθμητικο και ημερομηνιας αντιστοιχα

agn_data = agn_data.withColumn("Date", agn_data["Date"].cast("date"))
ainv_data = ainv_data.withColumn("Date", ainv_data["Date"].cast("date"))
ale_data = ale_data.withColumn("Date", ale_data["Date"].cast("date"))

In [73]:
# Κρατάμε μόνο τον μήνα απο την στήλη 'Date' και τον προσθέτουμε στο dataframe

agn_data_Month = agn_data.withColumn("Month", month("Date"))
ainv_data_Month = ainv_data.withColumn("Month", month("Date"))
ale_data_Month = ale_data.withColumn("Month", month("Date"))

In [74]:
# Ομαδοποιούμε τα δεδομένα ανα μήνα και υπολογίζουμε τον μέσο όρο των στηλών 'Open', 'Close' και 'Volume'

agn_avg = agn_data_Month.groupBy("Month").avg("Open", "Close", "Volume").orderBy("Month")
ainv_avg = ainv_data_Month.groupBy("Month").avg("Open", "Close", "Volume").orderBy("Month")
ale_avg = ale_data_Month.groupBy("Month").avg("Open", "Close", "Volume").orderBy("Month")

In [75]:
# Εμφανίζουμε τα αποτελέσματα
agn_avg.show()

+-----+------------------+------------------+------------------+
|Month|         avg(Open)|        avg(Close)|       avg(Volume)|
+-----+------------------+------------------+------------------+
|    1|101.83225246590216|101.80168585941709|         1796786.0|
|    2|  106.105923828125|106.37493228149414|       2018339.112|
|    3|108.97477564895362|108.91759996916119|1954400.0771929824|
|    4|104.96801150023047|104.85799636413802|2048475.3246268656|
|    5|106.58948712435635|106.51974534468218| 2020206.469090909|
|    6|110.05557827268328|110.13075361933026|1611504.9321428572|
|    7| 113.9869779440073|114.04540697034899|1486405.2857142857|
|    8|112.94372663530923|112.76380307286668|1592136.8719723183|
|    9|113.42711665935086|113.29473668292053|1720100.3007518798|
|   10|109.94360694885253|109.83696154544228| 2045412.403508772|
|   11|105.93957639400196|105.72473086481509| 2363413.675889328|
|   12| 107.0282400236355|107.06868900088813|1565874.4606299212|
+-----+------------------

In [76]:
# Εμφανίζουμε τα αποτελέσματα
ainv_avg.show()

+-----+------------------+-----------------+------------------+
|Month|         avg(Open)|       avg(Close)|       avg(Volume)|
+-----+------------------+-----------------+------------------+
|    1| 5.960922838741318|  5.9578900446041|2664324.3029045644|
|    2| 5.910714666391241|5.914387079148457| 3277507.870689655|
|    3| 6.044150871561285|6.036955775085248|3266939.4210526315|
|    4| 6.208691410609146|6.210719027625981| 2431237.593283582|
|    5|6.2512025330283425|6.252906892082908|2611925.7163636363|
|    6| 6.227297857829503|6.216372852666037|2798179.3964285715|
|    7| 6.111186066826622|6.118620138901931|2189656.0036630034|
|    8| 6.283973707872279|6.294057775946224| 2754273.910034602|
|    9| 6.352289081516123|6.345983075020008|2706609.6917293235|
|   10|  6.21246140546966|6.203042460324472|2141400.1192982458|
|   11| 6.114790127682592|  6.1033130511936|2523151.9881422925|
|   12| 5.926174418194088|5.918955116760074|2916861.6377952755|
+-----+------------------+--------------

In [77]:
# Εμφανίζουμε τα αποτελέσματα
ale_avg.show()

+-----+------------------+------------------+------------------+
|Month|         avg(Open)|        avg(Close)|       avg(Volume)|
+-----+------------------+------------------+------------------+
|    1| 37.59970537557641|37.633916973573044|223401.15352697094|
|    2|38.356318876661106| 38.34859057130485|        240881.875|
|    3| 38.04251227127878| 38.07297179238838| 253724.5649122807|
|    4| 38.59285437882836|  38.6146268986944|240209.06343283583|
|    5| 39.37950182134455|39.382316367409445|244737.14909090908|
|    6| 39.89017846924918| 39.91864985738482|257443.88214285715|
|    7| 40.26568859924764|40.257758206936906| 209357.9010989011|
|    8| 40.10728026261379|  40.0871074108929| 245987.4429065744|
|    9| 39.87034198933078|39.861116545540945|226992.70676691728|
|   10|40.066157906515556|40.064231524551126| 235927.1754385965|
|   11| 38.16249403369285|   38.125442693356| 225893.2648221344|
|   12| 37.39472042293999| 37.40590548327589| 234291.1968503937|
+-----+------------------

In [78]:
# Φιλτράρουμε τα δεδομένα όπου η τιμή 'Open' είναι πάνω από 35 και μετράμε τις εμφανίσεις

agn_over_35 = agn_data.filter(col("Open") > 35).count()
ainv_over_35 = ainv_data.filter(col("Open") > 35).count()
ale_over_35 = ale_data.filter(col("Open") > 35).count()

In [79]:
# Εμφανίζουμε τα αποτελέσματα

print(f"For AGN, the open price was over 35 for {agn_over_35} days.")
print(f"For AINV, the open price was over 35 for {ainv_over_35} days.")
print(f"For ALE, the open price was over 35 for {ale_over_35} days.")

For AGN, the open price was over 35 for 2071 days.
For AINV, the open price was over 35 for 0 days.
For ALE, the open price was over 35 for 1667 days.


In [80]:
# Βρίσκουμε την μέγιστη τιμή 'Open' για κάθε μετοχή

agn_max_open = agn_data.agg({"Open": "max"}).collect()[0][0]
ainv_max_open = ainv_data.agg({"Open": "max"}).collect()[0][0]
ale_max_open = ale_data.agg({"Open": "max"}).collect()[0][0]

# Φιλτράρουμε τα δεδομένα για να περιλαμβάνουν όλες τις ημερομηνίες με την υψηλότερη τιμή 'Open'

agn_dates_max_open = agn_data.filter(agn_data["Open"] == agn_max_open).select("Date").collect()
ainv_dates_max_open = ainv_data.filter(ainv_data["Open"] == ainv_max_open).select("Date").collect()
ale_dates_max_open = ale_data.filter(ale_data["Open"] == ale_max_open).select("Date").collect()


In [81]:
# Εμφανίζουμε τα αποτελέσματα

print(f"For AGN, the dates with the highest open price were:")
for row in agn_dates_max_open:
    print(row[0])

print(f"For AINV, the dates with the highest open price were:")
for row in ainv_dates_max_open:
    print(row[0])

print(f"For ALE, the dates with the highest open price were:")
for row in ale_dates_max_open:
    print(row[0])

For AGN, the dates with the highest open price were:
2015-07-30
For AINV, the dates with the highest open price were:
2007-02-20
For ALE, the dates with the highest open price were:
2017-11-01


In [82]:
# Βρίσκουμε την μέγιστη τιμή 'Volume' για κάθε μετοχή

agn_max_volume = agn_data.agg({"Volume": "max"}).collect()[0][0]
ainv_max_volume = ainv_data.agg({"Volume": "max"}).collect()[0][0]
ale_max_volume = ale_data.agg({"Volume": "max"}).collect()[0][0]

# Φιλτράρουμε τα δεδομένα για να περιλαμβάνουν όλες τις ημερομηνίες με την υψηλότερη τιμή 'Volume'

agn_dates_max_volume = agn_data.filter(agn_data["Volume"] == agn_max_volume).select("Date").collect()
ainv_dates_max_volume = ainv_data.filter(ainv_data["Volume"] == ainv_max_volume).select("Date").collect()
ale_dates_max_volume = ale_data.filter(ale_data["Volume"] == ale_max_volume).select("Date").collect()

In [83]:
# Εμφανίζουμε τα αποτελέσματα

print(f"For AGN, the dates with the highest volume were:")
for row in agn_dates_max_volume:
    print(row[0])

print(f"For AINV, the dates with the highest volume were:")
for row in ainv_dates_max_volume:
    print(row[0])

print(f"For ALE, the dates with the highest volume were:")
for row in ale_dates_max_volume:
    print(row[0])

For AGN, the dates with the highest volume were:
2016-04-05
For AINV, the dates with the highest volume were:
2014-02-28
For ALE, the dates with the highest volume were:
2014-02-27


In [84]:
# Βρίσκουμε την μέγιστη τιμή 'Close' για κάθε μετοχή

agn_data_year = agn_data.withColumn("Year", year("Date"))
ainv_data_year = ainv_data.withColumn("Year", year("Date"))
ale_data_year = ale_data.withColumn("Year", year("Date"))

In [85]:
# Βρίσκουμε την μέγιστη τιμή 'Open' για κάθε μετοχή
agn_max_open_year = agn_data_year.groupBy("Year").agg(max("Open").alias("MaxOpen")).orderBy("MaxOpen", ascending=False).first()["Year"]
ainv_max_open_year = ainv_data_year.groupBy("Year").agg(max("Open").alias("MaxOpen")).orderBy("MaxOpen", ascending=False).first()["Year"]
ale_max_open_year = ale_data_year.groupBy("Year").agg(max("Open").alias("MaxOpen")).orderBy("MaxOpen", ascending=False).first()["Year"]

# Βρίσκουμε την μέγιστη τιμή 'Close' για κάθε μετοχή
agn_max_close_year = agn_data_year.groupBy("Year").agg(max("Close").alias("MaxClose")).orderBy("MaxClose", ascending=False).first()["Year"]
ainv_max_close_year = ainv_data_year.groupBy("Year").agg(max("Close").alias("MaxClose")).orderBy("MaxClose", ascending=False).first()["Year"]
ale_max_close_year = ale_data_year.groupBy("Year").agg(max("Close").alias("MaxClose")).orderBy("MaxClose", ascending=False).first()["Year"]

In [86]:
# Εμφανίζουμε τα αποτελέσματα

print(f"For AGN, the year with the highest open price was: {agn_max_open_year}")
print(f"For AINV, the year with the highest open price was: {ainv_max_open_year}")
print(f"For ALE, the year with the highest open price was: {ale_max_open_year}")

print(f"For AGN, the year with the highest close price was: {agn_max_close_year}")
print(f"For AINV, the year with the highest close price was: {ainv_max_close_year}")
print(f"For ALE, the year with the highest close price was: {ale_max_close_year}")

For AGN, the year with the highest open price was: 2015
For AINV, the year with the highest open price was: 2007
For ALE, the year with the highest open price was: 2017
For AGN, the year with the highest close price was: 2015
For AINV, the year with the highest close price was: 2007
For ALE, the year with the highest close price was: 2017
